# Formula 1 Race Predictor
### CP468 Final Project
### By: Robert Mazza and Ronny Yehia

<img src="./Images/F1-logo.png" alt="F1 Logo" width="50%"/>

Data set used: https://www.kaggle.com/code/johnnymcclorey/f1-exploratory-analysis

See the README for more contextual information around this project.

# Data Collection and Cleaning

### Importing the data

In [90]:
import pandas as pd
import numpy as np

# Importing the datasets
results_df = pd.read_csv('./data/results.csv')
status_df = pd.read_csv('./data/status.csv')
drivers_df = pd.read_csv('./data/drivers.csv')
races_df = pd.read_csv('./data/races.csv')
constructor_df = pd.read_csv('./data/constructors.csv')
driver_standings_df = pd.read_csv('./data/driver_standings.csv')
quali_df = pd.read_csv('./data/qualifying.csv')
constructor_standings_df = pd.read_csv('./data/constructor_standings.csv')
pd.get_option("display.max_columns",None)


We are importing the necessary data sets along with the necessary packages to correctly assess the data. The above data sets include races, drivers, standings, results, qualifications for certain races, etc. 

## Qualifying
The rules of qualifying has changed over the years regarding how many sessions there are, and the time they last. The one variable that has never changed is how the fastest lap each driver sets counts towards where they start the race. So for this reason I will only be focusing on the starting position of each driver since that is the result of their qualifying performance.

<img src="./Images/starting-grid.jpg" width="50%"/>


In [91]:

# drop driver number column as it is not needed since we have the driver id
quali_df.drop(['number'], axis=1, inplace=True)

# drop Q1, Q2, Q3 columns as we only care about starting position
quali_df.drop(['q1','q2','q3'], axis=1, inplace=True)

# check for null values
print(quali_df.isna().sum())
# drop all rows that have NULL values
quali_df.dropna(inplace = True)

print((quali_df['qualifyId'] >= 0).all())
print((quali_df['raceId'] >= 0).all())
print((quali_df['driverId'] >= 0).all())
print((quali_df['constructorId'] >= 0).all())
print((quali_df['position'] >= 0).all())

print()


qualifyId        0
raceId           0
driverId         0
constructorId    0
position         0
dtype: int64
True
True
True
True
True



Prior to us even being able to predict which drivers are going to place where, we must see who qualifies. Starting position is based on the time set on the qualifying laps before the actual competitive race has begun. From there we can determine where each driver is starting, which will impact their chances in the race. 

In [92]:
quali_df.head()

,qualifyId,raceId,driverId,constructorId,position
0,1,18,1,1,1
1,2,18,9,2,2
2,3,18,5,1,3
3,4,18,13,6,4
4,5,18,2,2,5


## Races


In [93]:
# drop the time column as it is not needed
races_df.drop(['time'], axis=1, inplace=True)

# drop the wikipedia URLs column
races_df.drop(['url'], axis=1, inplace=True)

# check for null values
print(races_df.isna().sum())

races_df.info()


raceId       0
year         0
round        0
circuitId    0
name         0
date         0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1079 entries, 0 to 1078
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   raceId     1079 non-null   int64 
 1   year       1079 non-null   int64 
 2   round      1079 non-null   int64 
 3   circuitId  1079 non-null   int64 
 4   name       1079 non-null   object
 5   date       1079 non-null   object
dtypes: int64(4), object(2)
memory usage: 50.7+ KB


We can see from the sums above we have no null values.

After dropping unnecessary values that wouldn't be used to determine placement in the actual race, we double check for null values to ensure there are none

In [94]:
races_df.head()

,raceId,year,round,circuitId,name,date
0,1,2009,1,1,Australian Grand Prix,29/03/09
1,2,2009,2,2,Malaysian Grand Prix,05/04/09
2,3,2009,3,17,Chinese Grand Prix,19/04/09
3,4,2009,4,3,Bahrain Grand Prix,26/04/09
4,5,2009,5,4,Spanish Grand Prix,10/05/09


## Results


In [95]:
results_df.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [96]:
# check for null values
results_df.isna().sum()

resultId           0
raceId             0
driverId           0
constructorId      0
number             0
grid               0
position           0
positionText       0
positionOrder      0
points             0
laps               0
time               0
milliseconds       0
fastestLap         0
rank               0
fastestLapTime     0
fastestLapSpeed    0
statusId           0
dtype: int64

In [97]:
# dropping unneeded columns
results_df.drop(['time', 'number'], axis=1, inplace=True)

## Driver Standings

In [98]:
# check for null values
print(driver_standings_df.isna().sum())

driver_standings_df.info()

print("Checking if all values are equal or greater than zero")
# check if each coloumn that is int64 has a value greater or equal of zero
# True = good, False = bad
print((driver_standings_df['driverStandingsId'] >= 0).all())
print((driver_standings_df['raceId'] >= 0).all())
print((driver_standings_df['driverId'] >= 0).all())
print((driver_standings_df['points'] >= 0).all())
print((driver_standings_df['position'] >= 0).all())
print((driver_standings_df['wins'] >= 0).all())



driverStandingsId    0
raceId               0
driverId             0
points               0
position             0
positionText         0
wins                 0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33476 entries, 0 to 33475
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   driverStandingsId  33476 non-null  int64  
 1   raceId             33476 non-null  int64  
 2   driverId           33476 non-null  int64  
 3   points             33476 non-null  float64
 4   position           33476 non-null  int64  
 5   positionText       33476 non-null  object 
 6   wins               33476 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 1.8+ MB
Checking if all values are equal or greater than zero
True
True
True
True
True
True


As presumed, the driver standings are determined by numerous factors. All the values above that are checked whether they are larger or equal to 0 play a role in determining standings, points, position, wins, etc. All these values are taken into account when attempting to determine who is likely to place where in the race

In [99]:
driver_standings_df.head()

,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
1,2,18,2,8.0,2,2,0
2,3,18,3,6.0,3,3,0
3,4,18,4,5.0,4,4,0
4,5,18,5,4.0,5,5,0


## Constructors Standings

In [100]:
# check for null values
print(constructor_standings_df.isna().sum())

constructor_standings_df.info()

print("Checking if all values are equal or greater than zero")
# check if each coloumn that is int64 has a value greater or equal of zero
# True = good, False = bad
print((constructor_standings_df['constructorStandingsId'] >= 0).all())
print((constructor_standings_df['raceId'] >= 0).all())
print((constructor_standings_df['constructorId'] >= 0).all())
print((constructor_standings_df['points'] >= 0).all())
print((constructor_standings_df['position'] >= 0).all())
print((constructor_standings_df['wins'] >= 0).all())


constructorStandingsId    0
raceId                    0
constructorId             0
points                    0
position                  0
positionText              0
wins                      0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12741 entries, 0 to 12740
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   constructorStandingsId  12741 non-null  int64  
 1   raceId                  12741 non-null  int64  
 2   constructorId           12741 non-null  int64  
 3   points                  12741 non-null  float64
 4   position                12741 non-null  int64  
 5   positionText            12741 non-null  object 
 6   wins                    12741 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 696.9+ KB
Checking if all values are equal or greater than zero
True
True
True
True
True
True


In [101]:
constructor_standings_df.head()

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
0,1,18,1,14.0,1,1,1
1,2,18,2,8.0,3,3,0
2,3,18,3,9.0,2,2,0
3,4,18,4,5.0,4,4,0
4,5,18,5,2.0,5,5,0


## Constructors

In [102]:
# check for null values
print(constructor_df.isna().sum())

# drop URL column
constructor_df.drop(['url'], axis=1, inplace=True)
# drop constructor natonality column as it is not needed
constructor_df.drop(['nationality'], axis=1, inplace=True)

constructor_df.info()

print("Checking if all values are equal or greater than zero")
# check if each coloumn that has a value greater or equal of zero or not ''
# True = good, False = bad
print((constructor_df['name'] != '').all())
print((constructor_df['constructorRef'] != '').all())
print((constructor_df['constructorId'] >= 0).all())


constructorId     0
constructorRef    0
name              0
nationality       0
url               0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   constructorId   211 non-null    int64 
 1   constructorRef  211 non-null    object
 2   name            211 non-null    object
dtypes: int64(1), object(2)
memory usage: 5.1+ KB
Checking if all values are equal or greater than zero
True
True
True


In [103]:
constructor_df.head()

,constructorId,constructorRef,name
0,1,mclaren,McLaren
1,2,bmw_sauber,BMW Sauber
2,3,williams,Williams
3,4,renault,Renault
4,5,toro_rosso,Toro Rosso


## Drivers

In [104]:
# check for null values
print(drivers_df.isna().sum())
# drop URL column
drivers_df.drop(['url'], axis=1, inplace=True)
# drop constructor natonality column as it is not needed
drivers_df.drop(['nationality'], axis=1, inplace=True)
# drop driver number column, we will refer to each driver by their last name
drivers_df.drop(['number'], axis=1, inplace=True )

drivers_df.info()

print("Checking if all values are equal or greater than zero")
# check if each coloumn that has a value greater or equal of zero or not ''
# True = good, False = bad
print((drivers_df['driverRef'] != '').all())
print((drivers_df['code'] != '').all())
print((drivers_df['forename'] != '').all())
print((drivers_df['surname'] != '').all())
print((drivers_df['driverId'] >= 0).all())


driverId       0
driverRef      0
number         0
code           0
forename       0
surname        0
dob            0
nationality    0
url            0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   driverId   854 non-null    int64 
 1   driverRef  854 non-null    object
 2   code       854 non-null    object
 3   forename   854 non-null    object
 4   surname    854 non-null    object
 5   dob        854 non-null    object
dtypes: int64(1), object(5)
memory usage: 40.2+ KB
Checking if all values are equal or greater than zero
True
True
True
True
True


In [105]:
drivers_df.head()

,driverId,driverRef,code,forename,surname,dob
0,1,hamilton,HAM,Lewis,Hamilton,1985-01-07
1,2,heidfeld,HEI,Nick,Heidfeld,1977-05-10
2,3,rosberg,ROS,Nico,Rosberg,1985-06-27
3,4,alonso,ALO,Fernando,Alonso,1981-07-29
4,5,kovalainen,KOV,Heikki,Kovalainen,1981-10-19


## Status
Describes the status of the car in respect to finishing the race. If the car did not finish the race then it will have a "Status" that gives some more detail to why the car did not finish. 

In [106]:
# check for null values
print(status_df.isna().sum())

status_df.info()

print((status_df['statusId'] >= 0).all())
print((status_df['status'] != '').all())


statusId    0
status      0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   statusId  137 non-null    int64 
 1   status    137 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.3+ KB
True
True


In [107]:
status_df.head()

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


# Exploritory Analysis

# Data Preperation

Merging all the dataframes into a single comprehensive one.

In [108]:
# merging all seperate dataframe into single dataframe as df
df0 = pd.merge(results_df, quali_df, on = ['raceId', 'driverId', 'constructorId'])

df1 = pd.merge(df0, races_df, on ='raceId')

df2 = pd.merge(df1, drivers_df, on = 'driverId')

df3 = pd.merge(df2, driver_standings_df, on = 'driverId')

df4 = pd.merge(df3, constructor_df, on ='constructorId')

df = pd.merge(df4, status_df, on ='statusId')
pd.get_option("display.max_columns",None)

df.head()

,resultId,raceId_x,driverId,constructorId,grid,position_x,positionText_x,positionOrder,points_x,laps,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,qualifyId,position_y,year,round,circuitId,name_x,date,driverRef,code,forename,surname,dob,driverStandingsId,raceId_y,points_y,position,positionText_y,wins,constructorRef,name_y,status
0,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,16/03/08,hamilton,HAM,Lewis,Hamilton,1985-01-07,1,18,10.0,1,1,1,mclaren,McLaren,Finished
1,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,16/03/08,hamilton,HAM,Lewis,Hamilton,1985-01-07,9,19,14.0,1,1,1,mclaren,McLaren,Finished
2,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,16/03/08,hamilton,HAM,Lewis,Hamilton,1985-01-07,27,20,14.0,3,3,1,mclaren,McLaren,Finished
3,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,16/03/08,hamilton,HAM,Lewis,Hamilton,1985-01-07,48,21,20.0,2,2,1,mclaren,McLaren,Finished
4,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,16/03/08,hamilton,HAM,Lewis,Hamilton,1985-01-07,69,22,28.0,3,3,1,mclaren,McLaren,Finished


In [109]:
df.columns

Index(['resultId', 'raceId_x', 'driverId', 'constructorId', 'grid',
       'position_x', 'positionText_x', 'positionOrder', 'points_x', 'laps',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'qualifyId', 'position_y', 'year',
       'round', 'circuitId', 'name_x', 'date', 'driverRef', 'code', 'forename',
       'surname', 'dob', 'driverStandingsId', 'raceId_y', 'points_y',
       'position', 'positionText_y', 'wins', 'constructorRef', 'name_y',
       'status'],
      dtype='object')

In [110]:
col_name = {'milliseconds':'timetaken_in_millisec','fastestLapSpeed':'max_speed',
 'name_x':'grand_prix','number_y':'driver_num','code':'driver_code','name_y':'constructor_name',
 'raceId_x':'racerId','points_x':'points','position_y':'position'}

df.rename(columns=col_name, inplace=True)

df.head()

,resultId,racerId,driverId,constructorId,grid,position_x,positionText_x,positionOrder,points,laps,timetaken_in_millisec,fastestLap,rank,fastestLapTime,max_speed,statusId,qualifyId,position,year,round,circuitId,grand_prix,date,driverRef,driver_code,forename,surname,dob,driverStandingsId,raceId_y,points_y,position,positionText_y,wins,constructorRef,constructor_name,status
0,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,16/03/08,hamilton,HAM,Lewis,Hamilton,1985-01-07,1,18,10.0,1,1,1,mclaren,McLaren,Finished
1,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,16/03/08,hamilton,HAM,Lewis,Hamilton,1985-01-07,9,19,14.0,1,1,1,mclaren,McLaren,Finished
2,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,16/03/08,hamilton,HAM,Lewis,Hamilton,1985-01-07,27,20,14.0,3,3,1,mclaren,McLaren,Finished
3,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,16/03/08,hamilton,HAM,Lewis,Hamilton,1985-01-07,48,21,20.0,2,2,1,mclaren,McLaren,Finished
4,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,16/03/08,hamilton,HAM,Lewis,Hamilton,1985-01-07,69,22,28.0,3,3,1,mclaren,McLaren,Finished


In [111]:
# Combining the forename and surname of the driver into a single column
df['driver_name'] = df['forename'] + ' ' + df['surname']
df  = df.drop(['forename', 'surname'], axis=1)

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1559704 entries, 0 to 1559703
Data columns (total 36 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   resultId               1559704 non-null  int64  
 1   racerId                1559704 non-null  int64  
 2   driverId               1559704 non-null  int64  
 3   constructorId          1559704 non-null  int64  
 4   grid                   1559704 non-null  int64  
 5   position_x             1559704 non-null  object 
 6   positionText_x         1559704 non-null  object 
 7   positionOrder          1559704 non-null  int64  
 8   points                 1559704 non-null  float64
 9   laps                   1559704 non-null  int64  
 10  timetaken_in_millisec  1559704 non-null  object 
 11  fastestLap             1559704 non-null  object 
 12  rank                   1559704 non-null  object 
 13  fastestLapTime         1559704 non-null  object 
 14  max_speed         

Date and dob columns are being stored as an Object, so it will be changed to Date object below.

In [113]:
pd.to_datetime(df.date)

0         2008-03-16
1         2008-03-16
2         2008-03-16
3         2008-03-16
4         2008-03-16
             ...    
1559699   2021-12-12
1559700   2021-12-12
1559701   2021-12-12
1559702   2021-12-12
1559703   2021-12-12
Name: date, Length: 1559704, dtype: datetime64[ns]

In [114]:
df['dob'] = pd.to_datetime(df['dob'])
df['date'] = pd.to_datetime(df['date'])

In [115]:
from datetime import datetime

# getting each driver's age and storing it in a new column 'age'

dates = datetime.today()-df['dob']
age = dates.dt.days/365
# round them up if they are close to their next birthday than their last
df['age'] = round(age) 
pd.set_option('display.max_columns', None)
df.head()

,resultId,racerId,driverId,constructorId,grid,position_x,positionText_x,positionOrder,points,laps,timetaken_in_millisec,fastestLap,rank,fastestLapTime,max_speed,statusId,qualifyId,position,year,round,circuitId,grand_prix,date,driverRef,driver_code,dob,driverStandingsId,raceId_y,points_y,position,positionText_y,wins,constructorRef,constructor_name,status,driver_name,age
0,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,2008-03-16,hamilton,HAM,1985-01-07,1,18,10.0,1,1,1,mclaren,McLaren,Finished,Lewis Hamilton,38.0
1,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,2008-03-16,hamilton,HAM,1985-01-07,9,19,14.0,1,1,1,mclaren,McLaren,Finished,Lewis Hamilton,38.0
2,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,2008-03-16,hamilton,HAM,1985-01-07,27,20,14.0,3,3,1,mclaren,McLaren,Finished,Lewis Hamilton,38.0
3,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,2008-03-16,hamilton,HAM,1985-01-07,48,21,20.0,2,2,1,mclaren,McLaren,Finished,Lewis Hamilton,38.0
4,1,18,1,1,1,1,1,1,10.0,58,5690616,39,2,1:27.452,218.300,1,1,1,2008,1,1,Australian Grand Prix,2008-03-16,hamilton,HAM,1985-01-07,69,22,28.0,3,3,1,mclaren,McLaren,Finished,Lewis Hamilton,38.0


# Machine Learning Model